In [ ]:
library(monocle3)
library(destiny)
# Plot
library(conflicted)
library(purrr)

library(viridis)
library(cowplot)
library(ggplot2)
library(ggbeeswarm)
library(ggthemes)
library(SeuratData)
library(SeuratWrappers)
library(ggplot2)
library(patchwork)
library(magrittr)
library(Seurat)
library(data.table)
library(ggplot2)
library(readxl)
library(plyr)

library(dplyr)

library(reshape2)

library(viridis)
library(knitr)
library(stringr)
library(NMF)
library(rsvd)
library(lme4)
library(RColorBrewer)

library(gtools)
library(gprofiler2)

In [ ]:

setwd(".../Test_Code")
datasets <- readRDS(".../RData/SEZ_seurat_object.rds")
DefaultAssay(datasets) <- "RNA"


In [ ]:
colnames(datasets@meta.data)

In [ ]:
Idents(datasets) <- "group"
datasets_Y <- subset(datasets, idents = c("Y"))

In [ ]:
datasets_M <- subset(datasets, idents = c("M"))

In [ ]:
DefaultAssay(datasets_Y) <- "integrated"
DefaultAssay(datasets_M) <- "integrated"

In [ ]:
Idents(datasets_Y) <- datasets$cluster
Idents(datasets_M) <- datasets$cluster

In [ ]:
### Building the necessary parts for a basic cds

# part one, gene annotations

gene_annotation <- as.data.frame(rownames(datasets_Y@reductions[["pca"]]@feature.loadings), row.names = rownames(datasets_Y@reductions[["pca"]]@feature.loadings))
colnames(gene_annotation) <- "gene_short_name"


In [ ]:
# part two, cell information

cell_metadata <- as.data.frame(datasets_Y@assays[["RNA"]]@counts@Dimnames[[2]], row.names = datasets_Y@assays[["RNA"]]@counts@Dimnames[[2]])
colnames(cell_metadata) <- "barcode"



In [ ]:
# part three, counts sparse matrix

New_matrix <- datasets_Y@assays[["RNA"]]@counts
New_matrix <- New_matrix[rownames(datasets_Y@reductions[["pca"]]@feature.loadings), ]
expression_matrix <- New_matrix

In [ ]:
### Construct the basic cds object

cds_from_seurat <- new_cell_data_set(expression_matrix,
                                     cell_metadata = cell_metadata,
                                     gene_metadata = gene_annotation)


In [ ]:
### Construct and assign the made up partition

recreate.partition <- c(rep(1, length(cds_from_seurat@colData@rownames)))
names(recreate.partition) <- cds_from_seurat@colData@rownames
recreate.partition <- as.factor(recreate.partition)

cds_from_seurat@clusters@listData[["UMAP"]][["partitions"]] <- recreate.partition

In [ ]:
### Assign the cluster info

list_cluster <- datasets_Y@meta.data[["integrated_snn_res.0.4"]]
names(list_cluster) <- datasets_Y@assays[["RNA"]]@data@Dimnames[[2]]
cds_from_seurat@clusters@listData[["UMAP"]][["clusters"]] <- list_cluster


In [ ]:
### Could be a space-holder, but essentially fills out louvain parameters

cds_from_seurat@clusters@listData[["UMAP"]][["louvain_res"]] <- "NA"


### Assign UMAP coordinate
# cds_from_seurat <- preprocess_cds (cds_from_seurat,method = "PCA")
# cds_from_seurat <- reduce_dimension(cds_from_seurat)
reducedDims(cds_from_seurat)$UMAP <-  datasets_Y@reductions[["umap"]]@cell.embeddings


In [ ]:
### Assign feature loading for downstream module analysis

cds_from_seurat@reduce_dim_aux$gene_loadings <- datasets_Y@reductions[["pca"]]@feature.loadings

In [ ]:
### Learn graph, this step usually takes a significant period of time for larger samples

# print("Learning graph, which can take a while depends on the sample")

cds_from_seurat <- learn_graph(cds_from_seurat, use_partition = TRUE)

In [ ]:
ucols_all<-c("#993399", "#FFCCFF", '#CC0099' ,
             "#999999",
             "#FFFF00", 
             "#333333",'#3333FF',"#33CCFF","#99FFFF",#"#9933FF",'#66CC99',"#009966","#006633",
             '#99FF33', "#999900", "#CCCC66", "#666633", 
             "#9966CC", 
             "#663333",
             "#FF6600",
             "#669999",
             "#FF0000","#FFCC99","#FF9933") #ex SST xcolor "#99CC99"

In [ ]:
plot_cells(cds_from_seurat, label_groups_by_cluster = TRUE, label_leaves = TRUE, label_branch_points = TRUE,color_cells_by = 'cluster')+
scale_color_manual(values=ucols_all)


In [ ]:
plot_cells(cds_from_seurat, label_groups_by_cluster = TRUE, label_leaves = TRUE, label_branch_points = TRUE,color_cells_by = 'cluster')+
scale_color_manual(values=ucols_all)


In [ ]:
tiff("UMAP_trajectory_monocle3_Y.tiff", res = 400, units = "in", width = 5, height = 5)
plot_cells(cds_from_seurat, label_groups_by_cluster = TRUE, label_principal_points = TRUE,label_leaves = TRUE, label_branch_points = TRUE,color_cells_by = 'cluster')+
scale_color_manual(values=ucols_all)
dev.off()

In [ ]:
#we are going to use the ones that we think are stem cells!

In [ ]:
cds_from_seurat <- order_cells(cds_from_seurat,root_pr_nodes=c("Y_572","Y_544","Y_506","Y_205","Y_59",
                                                              "Y_175",
                                                              "Y_110",
                                                              "Y_414","Y_170","Y_71"))#
#aNCS,OPCs, clusters 0,7 and 16
cds_from_seurat <- order_cells(cds_from_seurat,root_pr_nodes=c("Y_59",
                                                              "Y_175",
                                                              "Y_110",
                                                              "Y_414","Y_170"))#
#only the stem clusters? 
#0 and 7

In [ ]:
plot_cells(cds_from_seurat, color_cells_by = "pseudotime", label_cell_groups = FALSE, label_leaves = FALSE, 
    label_branch_points = FALSE)
tiff("UMAP_pseudotime_monocle3_Y.tiff", res = 400, units = "in", width = 6, height = 5)

plot_cells(cds_from_seurat, color_cells_by = "pseudotime", label_cell_groups = FALSE, label_leaves = FALSE, 
    label_branch_points = FALSE)
dev.off()

In [ ]:
#dataset_M

In [ ]:
### Building the necessary parts for a basic cds

# part one, gene annotations

gene_annotation <- as.data.frame(rownames(datasets_M@reductions[["pca"]]@feature.loadings), row.names = rownames(datasets_M@reductions[["pca"]]@feature.loadings))
colnames(gene_annotation) <- "gene_short_name"


In [ ]:
# part two, cell information

cell_metadata <- as.data.frame(datasets_M@assays[["RNA"]]@counts@Dimnames[[2]], row.names = datasets_M@assays[["RNA"]]@counts@Dimnames[[2]])
colnames(cell_metadata) <- "barcode"



In [ ]:
# part three, counts sparse matrix

New_matrix <- datasets_M@assays[["RNA"]]@counts
New_matrix <- New_matrix[rownames(datasets_M@reductions[["pca"]]@feature.loadings), ]
expression_matrix <- New_matrix

In [ ]:
### Construct the basic cds object

cds_from_seurat <- new_cell_data_set(expression_matrix,
                                     cell_metadata = cell_metadata,
                                     gene_metadata = gene_annotation)


In [ ]:
### Construct and assign the made up partition

recreate.partition <- c(rep(1, length(cds_from_seurat@colData@rownames)))
names(recreate.partition) <- cds_from_seurat@colData@rownames
recreate.partition <- as.factor(recreate.partition)

cds_from_seurat@clusters@listData[["UMAP"]][["partitions"]] <- recreate.partition

In [ ]:
### Assign the cluster info

list_cluster <- datasets_M@meta.data[["integrated_snn_res.0.4"]]
names(list_cluster) <- datasets_M@assays[["RNA"]]@data@Dimnames[[2]]
cds_from_seurat@clusters@listData[["UMAP"]][["clusters"]] <- list_cluster


In [ ]:
### Could be a space-holder, but essentially fills out louvain parameters

cds_from_seurat@clusters@listData[["UMAP"]][["louvain_res"]] <- "NA"


### Assign UMAP coordinate
# cds_from_seurat <- preprocess_cds (cds_from_seurat,method = "PCA")
# cds_from_seurat <- reduce_dimension(cds_from_seurat)
reducedDims(cds_from_seurat)$UMAP <-  datasets_M@reductions[["umap"]]@cell.embeddings


In [ ]:
### Assign feature loading for downstream module analysis

cds_from_seurat@reduce_dim_aux$gene_loadings <- datasets_M@reductions[["pca"]]@feature.loadings

In [ ]:
### Learn graph, this step usually takes a significant period of time for larger samples

print("Learning graph, which can take a while depends on the sample")

cds_from_seurat <- learn_graph(cds_from_seurat, use_partition = TRUE)

In [ ]:
ucols_all<-c("#993399", "#FFCCFF", '#CC0099' ,
             "#999999",
             "#FFFF00", 
             "#333333",'#3333FF',"#33CCFF","#99FFFF",#"#9933FF",'#66CC99',"#009966","#006633",
             '#99FF33', "#999900", "#CCCC66", "#666633", 
             "#9966CC", 
             "#663333",
             "#FF6600",
             "#669999",
             "#FF0000","#FFCC99","#FF9933") #ex SST xcolor "#99CC99"

In [ ]:
plot_cells(cds_from_seurat, label_groups_by_cluster = TRUE, label_leaves = TRUE, label_branch_points = TRUE,color_cells_by = 'cluster')+
scale_color_manual(values=ucols_all)


In [ ]:
plot_cells(cds_from_seurat, label_groups_by_cluster = TRUE, label_leaves = TRUE, label_branch_points = TRUE,color_cells_by = 'cluster')+
scale_color_manual(values=ucols_all)


In [ ]:
tiff("UMAP_trajectory_monocle3_M.tiff", res = 400, units = "in", width = 5, height = 5)
plot_cells(cds_from_seurat, label_groups_by_cluster = TRUE, label_principal_points = TRUE,label_leaves = TRUE, label_branch_points = TRUE,color_cells_by = 'cluster')+
scale_color_manual(values=ucols_all)
dev.off()

In [ ]:
cds_from_seurat <- order_cells(cds_from_seurat,root_pr_nodes=c("Y_353","Y_166",
                                                              "Y_412","Y_217","Y_151",
                                                              "Y_14","Y_522",
                                                              "Y_517","Y_132",
                                                              "Y_306","Y_324"))#,"Y_572","Y_544"
#cluster 7 and 16

In [ ]:
plot_cells(cds_from_seurat, color_cells_by = "pseudotime", label_cell_groups = FALSE, label_leaves = FALSE, 
    label_branch_points = FALSE)
tiff("UMAP_pseudotime_monocle3_M.tiff", res = 400, units = "in", width = 6, height = 5)

plot_cells(cds_from_seurat, color_cells_by = "pseudotime", label_cell_groups = FALSE, label_leaves = FALSE, 
    label_branch_points = FALSE)
dev.off()

In [ ]:
getwd()

In [ ]:
#complete datasets

In [ ]:
### Building the necessary parts for a basic cds

# part one, gene annotations

gene_annotation <- as.data.frame(rownames(datasets@reductions[["pca"]]@feature.loadings), row.names = rownames(datasets@reductions[["pca"]]@feature.loadings))
colnames(gene_annotation) <- "gene_short_name"


In [ ]:
# part two, cell information

cell_metadata <- as.data.frame(datasets@assays[["RNA"]]@counts@Dimnames[[2]], row.names = datasets@assays[["RNA"]]@counts@Dimnames[[2]])
colnames(cell_metadata) <- "barcode"



In [ ]:
# part three, counts sparse matrix

New_matrix <- datasets@assays[["RNA"]]@counts
New_matrix <- New_matrix[rownames(datasets@reductions[["pca"]]@feature.loadings), ]
expression_matrix <- New_matrix

In [ ]:
### Construct the basic cds object

cds_from_seurat <- new_cell_data_set(expression_matrix,
                                     cell_metadata = cell_metadata,
                                     gene_metadata = gene_annotation)


In [ ]:
### Construct and assign the made up partition

recreate.partition <- c(rep(1, length(cds_from_seurat@colData@rownames)))
names(recreate.partition) <- cds_from_seurat@colData@rownames
recreate.partition <- as.factor(recreate.partition)

cds_from_seurat@clusters@listData[["UMAP"]][["partitions"]] <- recreate.partition

In [ ]:
### Assign the cluster info

list_cluster <- datasets@meta.data[["integrated_snn_res.0.4"]]
names(list_cluster) <- datasets@assays[["RNA"]]@data@Dimnames[[2]]
cds_from_seurat@clusters@listData[["UMAP"]][["clusters"]] <- list_cluster


In [ ]:
### Could be a space-holder, but essentially fills out louvain parameters

cds_from_seurat@clusters@listData[["UMAP"]][["louvain_res"]] <- "NA"


### Assign UMAP coordinate
# cds_from_seurat <- preprocess_cds (cds_from_seurat,method = "PCA")
# cds_from_seurat <- reduce_dimension(cds_from_seurat)
reducedDims(cds_from_seurat)$UMAP <-  datasets@reductions[["umap"]]@cell.embeddings


In [ ]:
### Assign feature loading for downstream module analysis

cds_from_seurat@reduce_dim_aux$gene_loadings <- datasets@reductions[["pca"]]@feature.loadings

In [ ]:
### Learn graph, this step usually takes a significant period of time for larger samples

print("Learning graph, which can take a while depends on the sample")

cds_from_seurat <- learn_graph(cds_from_seurat, use_partition = TRUE)

In [ ]:
ucols_all<-c("#993399", "#FFCCFF", '#CC0099' ,
             "#999999",
             "#FFFF00", 
             "#333333",'#3333FF',"#33CCFF","#99FFFF",#"#9933FF",'#66CC99',"#009966","#006633",
             '#99FF33', "#999900", "#CCCC66", "#666633", 
             "#9966CC", 
             "#663333",
             "#FF6600",
             "#669999",
             "#FF0000","#FFCC99","#FF9933") #ex SST xcolor "#99CC99"

In [ ]:
plot_cells(cds_from_seurat, label_groups_by_cluster = TRUE, label_leaves = TRUE, label_branch_points = TRUE,color_cells_by = 'cluster')+
scale_color_manual(values=ucols_all)


In [ ]:
plot_cells(cds_from_seurat, label_groups_by_cluster = TRUE, label_leaves = TRUE, label_branch_points = TRUE,color_cells_by = 'cluster')+
scale_color_manual(values=ucols_all)


In [ ]:
tiff("UMAP_trajectory_monocle3_all_cells.tiff", res = 400, units = "in", width = 5, height = 5)
plot_cells(cds_from_seurat, label_groups_by_cluster = TRUE, label_principal_points = TRUE,label_leaves = TRUE, label_branch_points = TRUE,color_cells_by = 'cluster')+
scale_color_manual(values=ucols_all)
dev.off()

In [ ]:
cds_from_seurat <- order_cells(cds_from_seurat,root_pr_nodes=c("Y_268","Y_116","Y_436",
                                                              "Y_86","Y_93","Y_555",
                                                              "Y_579","Y_226","Y_398",
                                                              "Y_630","Y_545"))#,"Y_572","Y_544"



In [ ]:
plot_cells(cds_from_seurat, color_cells_by = "pseudotime", label_cell_groups = FALSE, label_leaves = FALSE, 
    label_branch_points = FALSE)
tiff("UMAP_pseudotime_monocle3_all_cells.tiff", res = 400, units = "in", width = 6, height = 5)

plot_cells(cds_from_seurat, color_cells_by = "pseudotime", label_cell_groups = FALSE, label_leaves = FALSE, 
    label_branch_points = FALSE)
dev.off()